# Обучение моделей и анализ результатов

## Что было сделано в предыдущем ноутбуке

В предыдущем ноутбуке был выполнен полный цикл подготовки данных для задачи классификации. В частности, были реализованы следующие этапы:

- сбор данных из нескольких источников (lenta.ru и РИА Новости);
- очистка данных от некорректных и дублирующихся примеров;
- удаление пустых и слишком коротких текстов;
- приведение тематик к единому числовому формату;
- объединение данных из разных источников в единый датасет;
- базовый анализ данных, включая:
  - распределение текстов по тематикам,
  - анализ длины текстов,
  - визуальную проверку качества текстов на примерах.

Таким образом, на вход текущего ноутбука подаётся уже очищенный, провалидированный и проанализированный датасет, готовый к использованию.

## Эксперименты с моделями

В процессе работы было проведено большое количество экспериментов с целью улучшения качества модели. Основное внимание уделялось подбору параметров текстовой векторизации и выбору модели классификации.

В частности, были протестированы различные варианты векторизации текста:
- `CountVectorizer` с разными параметрами,
- `TfidfVectorizer` с изменением:
  - диапазона n-грамм,
  - порогов `min_df` и `max_df`,
  - использования стоп-слов.

Также рассматривались **разные линейные модели для классификации**, в том числе линейная регрессия и логистическая регрессия. По результатам экспериментов было выявлено, что **логистическая регрессия стабильно показывает лучшие результаты качества**, особенно в задаче мультиклассовой классификации.

## Итоговая модель
 В результате экспериментов было принято решение использовать не одну отдельную модель, а ансамбль, построенный на основе двух        подходов к векторизации текста:
- `CountVectorizer`,
- `TfidfVectorizer`.

Для обеих векторизаций были подобраны оптимальные параметры, а в качестве классификатора в обоих случаях использовалась логистическая регрессия. Финальное предсказание формируется путём усреднения вероятностей, полученных от двух моделей.

Данный ансамблевый подход показал наилучшее качество по сравнениюс одиночными моделями и был выбран в качестве финального решения.

In [10]:
# Импортирую библиотеки

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score
import nltk
from nltk.corpus import stopwords
import numpy as np
import gdown

In [11]:
# train слишком большой, чтобы использовать
file_id = "1E3qzqIUtpaMneafD7qpQZjMC5LjylonV"
gdown.download(
    f"https://drive.google.com/uc?id={file_id}",
    "train_news.csv",
    quiet=False
)

url_test = "https://drive.google.com/uc?id=1H-p8uXFEeHv20q7Yik7wTNvmwkKNf3dS"
url_submission = "https://drive.google.com/uc?id=1ZdYPEZr5ohDb1oLqe8sv3uGwyhJ_B1LI"


train = pd.read_csv("train_news.csv")
# train = pd.read_csv(url_train)

# test = pd.read_csv("test_news.csv")
test = pd.read_csv(url_test)

# submit = pd.read_csv("base_submission_news.csv")
submit = pd.read_csv(url_submission)

display(train.head())
display(test.head())

Downloading...
From (original): https://drive.google.com/uc?id=1E3qzqIUtpaMneafD7qpQZjMC5LjylonV
From (redirected): https://drive.google.com/uc?id=1E3qzqIUtpaMneafD7qpQZjMC5LjylonV&confirm=t&uuid=94e8f7fc-01f8-4da9-9ca0-e6d72733c2e5
To: /content/train_news.csv
100%|██████████| 1.42G/1.42G [00:11<00:00, 126MB/s]


,content,topic
0,россиянам дали советы при выборе чая. рекоменд...,0
1,спикер госдумы вячеслав володин назвал угрозой...,0
2,в москве полицейские застрелили мужчину при по...,2
3,в загсе казани официально вступила в брак пара...,0
4,россиян предостерегли от нового способа кражи ...,0


,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...


In [13]:
# Разделение данных на признаки и целевую переменную

X = train["content"]
y = train["topic"]

# Делим данные на обучающую и валидационную выборки
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=17, stratify=y)

In [14]:
# Подготовка стоп-слов
# Загружаем список русскоязычных стоп-слов из NLTK
nltk.download('stopwords')
russian_stopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
pipe_count = Pipeline([
    ("vec", CountVectorizer(ngram_range=(1, 1))),
    ("scaler", MaxAbsScaler()),

    # Логистическая регрессия для мультиклассовой классификации
    ("clf", LogisticRegression(
        max_iter=500,
        class_weight="balanced",  # учитываем дисбаланс классов
        random_state=17))
])

In [16]:
pipe_tfidf = Pipeline([
    ("vec", TfidfVectorizer(
        min_df=5,                    # убираем редкие слова
        max_df=0.7,                  # убираем слишком частые слова
        ngram_range=(1, 3),          # используем уни-, би- и триграммы
        stop_words=russian_stopwords # удаляем стоп-слова
    )),
    ("scaler", MaxAbsScaler()),
    ("clf", LogisticRegression(
        max_iter=200,
        class_weight="balanced",  # учитываем дисбаланс классов
        random_state=17))
])

In [17]:
# Обучение обеих моделей
pipe_count.fit(x_train, y_train)
pipe_tfidf.fit(x_train, y_train)

# Получение вероятностей на валидации
proba_count = pipe_count.predict_proba(x_val)
proba_tfidf = pipe_tfidf.predict_proba(x_val)

In [18]:
# Сохраняем порядок классов
classes = pipe_count.named_steps["clf"].classes_

alpha = 0.5  # веса можно варьировать
proba_ens = alpha * proba_count + (1 - alpha) * proba_tfidf
y_val_ens = classes[proba_ens.argmax(axis=1)]

# Оценка качества моделей
print("Отдельно Count:")
print(classification_report(y_val, pipe_count.predict(x_val)))

print("Отдельно TF-IDF:")
print(classification_report(y_val, pipe_tfidf.predict(x_val)))

print("Ансамбль Count + TF-IDF:")
print(classification_report(y_val, y_val_ens))
print("macro F1 ансамбля:", f1_score(y_val, y_val_ens, average="macro"))

Отдельно Count:
              precision    recall  f1-score   support

           0       0.91      0.86      0.89     41155
           1       0.90      0.92      0.91     19904
           2       0.90      0.93      0.92     13215
           3       0.88      0.92      0.90     20548
           4       0.99      0.99      0.99      9230
           5       0.92      0.97      0.94      4159
           6       0.67      0.70      0.69      1418
           7       0.95      0.96      0.95      7297
           8       0.92      0.94      0.93      9002

    accuracy                           0.91    125928
   macro avg       0.89      0.91      0.90    125928
weighted avg       0.91      0.91      0.91    125928

Отдельно TF-IDF:
              precision    recall  f1-score   support

           0       0.92      0.90      0.91     41155
           1       0.92      0.93      0.93     19904
           2       0.93      0.93      0.93     13215
           3       0.91      0.93      0.92  

## Анализ результатов и вывод

В ходе экспериментов были сравненены три подхода: модель на `CountVectorizer`, модель на `TF-IDF` и их ансамбль.

- CountVectorizer + LogisticRegression показал хороший базовый результат: accuracy = 0.91, macro F1 = 0.90.  
  При этом хуже всего модель справляется с самым редким классом Строительство: F1 = 0.69.

- TF-IDF + LogisticRegression оказался лучшим из одиночных решений: accuracy = 0.93, macro F1 = 0.92.  
  Особенно заметно улучшение по классу Строительство: F1 вырос до 0.74. В целом TF-IDF более устойчиво работает на разных тематиках.

- Ансамбль Count + TF-IDF дал качество между двумя моделями: accuracy = 0.92, macro F1 = 0.91.  
  По сравнению с TF-IDF ансамбль не улучшил итоговую метрику, то есть в данной конфигурации добавление Count-модели не принесло прироста.

### Вывод
Лучший результат показала модель TF-IDF + Logistic Regression.  
Вероятная причина, что TF-IDF снижает влияние слишком частых слов и лучше выделяет информативные термины.

In [21]:
# Обучение моделей на полном обучающем датасете

X_full = train["content"]
y_full = train["topic"]
X_test_full = test["content"]

pipe_count.fit(X_full, y_full)
pipe_tfidf.fit(X_full, y_full)

Pipeline(steps=[('vec',
                 TfidfVectorizer(max_df=0.7, min_df=5, ngram_range=(1, 3),
                                 stop_words=['и', 'в', 'во', 'не', 'что', 'он',
                                             'на', 'я', 'с', 'со', 'как', 'а',
                                             'то', 'все', 'она', 'так', 'его',
                                             'но', 'да', 'ты', 'к', 'у', 'же',
                                             'вы', 'за', 'бы', 'по', 'только',
                                             'ее', 'мне', ...])),
                ('scaler', MaxAbsScaler()),
                ('clf',
                 LogisticRegression(class_weight='balanced', max_iter=200,
                                    random_state=17))])

In [22]:
# Предсказания CountVectorizer

proba_count_test = pipe_count.predict_proba(X_test_full)
preds_count = classes[proba_count_test.argmax(axis=1)]

submit_count = submit.copy()
submit_count["topic"] = preds_count
submit_count.to_csv("submission_count.csv", index=False)

In [23]:
# Предсказания TF-IDF

proba_tfidf_test = pipe_tfidf.predict_proba(X_test_full)
preds_tfidf = classes[proba_tfidf_test.argmax(axis=1)]

submit_tfidf = submit.copy()
submit_tfidf["topic"] = preds_tfidf
submit_tfidf.to_csv("submission_tfidf.csv", index=False)

In [24]:
# Ансамбль (усреднение вероятностей)

proba_ens_test = alpha * proba_count_test + (1 - alpha) * proba_tfidf_test
preds_ens = classes[proba_ens_test.argmax(axis=1)]

submit_ens = submit.copy()
submit_ens["topic"] = preds_ens
submit_ens.to_csv("submission_ensemble.csv", index=False)

## Финальный выбор модели

После обучения моделей и формирования отдельных сабмитов для
`CountVectorizer`, `TF-IDF` и их ансамбля, все три варианта были
протестированы на платформе Kaggle.

По результатам оценки на тестовых данных оказалось, что **ансамбль
моделей (Count + TF-IDF)** показал **наилучшее качество среди всех
рассмотренных вариантов**, превзойдя одиночные модели по итоговой
метрике на сабмишене.

submission_tfidf.csv - 0.89701

submission_count.csv - 0.86679

submission_ensemble.csv - 0.90039


Таким образом, несмотря на то, что на валидационной выборке ансамбль
не всегда демонстрировал явное преимущество, на скрытых тестовых
данных его использование оказалось оправданным.

В качестве финального решения для сдачи был выбран именно ансамбль,
так как он показал наилучший результат на тестовой выборке.

## Возможные улучшения

В ходе работы рассматривалась возможность использования других моделей, поддерживающих мультиклассовую классификацию, таких как SVM, градиентный бустинг и другие линейные и нелинейные алгоритмы. Однако из-за ограничений по времени и близости дедлайна было принято решение сосредоточиться на линейных моделях, которые показали стабильное и воспроизводимое качество.

Также в рамках данного задания не использовались инструменты автоматического подбора гиперпараметров, такие как Optuna. Подбор параметров выполнялся вручную на основе экспериментов. Использование Optuna могло бы позволить более эффективно исследовать пространство параметров и, возможно, дополнительно улучшить качество модели.

В дальнейшем, при наличии дополнительного времени, планируется протестировать другие модели и применить автоматический подбор гиперпараметров. В рамках данной работы используется ансамбль моделей на основе логистической регрессии, так как он показал наилучший результат среди протестированных подходов.

Так же можно разширить временной диапазон для наших данных. Брать не временной промежуток с 2023 года по 2025, а взять еще года ранее